In [1]:
from lib.kalman_filter import generate_kalmanfilter,apply_kalmanfilter
from lib.io import load_pickle_data
from lib.noglobal import noglobal


from external_lib.evaluation_function import calc_haversine


import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
train_path = "/work/data/input/selfmade_dataset/baseline_with_derived_data_v4/train.pkl"
train_df = load_pickle_data(train_path)


### カルマンフィルターを行う前

In [3]:
calc_haversine(train_df["latDeg_gt"],train_df["lngDeg_gt"],train_df["latDeg"],train_df["lngDeg"]).mean()

3.8468553394627825

### カルマンフィルター１回実施後

In [4]:


ret_list = [];


for key,each_df in tqdm(train_df.groupby("phone")):
    kf = generate_kalmanfilter()    
    result = apply_kalmanfilter(each_df[["latDeg","lngDeg"]].to_numpy(),kf)    
    each_df[ ["latDeg_kf","lngDeg_kf"]] = result
    ret_list.append(each_df)
    
ret = pd.concat(ret_list);

  0%|          | 0/73 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
calc_haversine(ret["latDeg_gt"],ret["lngDeg_gt"],ret["latDeg_kf"],ret["lngDeg_kf"]).mean()

## カルマンフィルター２回実施後

In [ ]:
ret_list = [];

ret_df = ret.copy()

for key,each_df in tqdm(ret_df.groupby("phone")):
    kf = generate_kalmanfilter()    
    result = apply_kalmanfilter(each_df[["latDeg_kf","lngDeg_kf"]].to_numpy(),kf)    
    each_df[["latDeg_kf_2","lngDeg_kf_2"]] = result    
    ret_list.append(each_df)
    
ret = pd.concat(ret_list);

In [ ]:
calc_haversine(ret["latDeg_gt"],ret["lngDeg_gt"],ret["latDeg_kf_2"],ret["lngDeg_kf_2"]).mean()

In [ ]:
current_df = train_df.copy()
cols = train_df.columns.values

for _ in range(1,6):
    ret_list = [];
    
    for key,each_df in tqdm(current_df.groupby("phone")):    
        kf = generate_kalmanfilter();        
        result = apply_kalmanfilter(each_df[["latDeg","lngDeg"]].to_numpy(),kf);
        each_df[["latDeg_kf","lngDeg_kf"]] = result
        ret_list.append(each_df)
        
    ret = pd.concat(ret_list)
            
    #display(calc_haversine(ret["latDeg_gt"],ret["lngDeg_gt"],ret["latDeg_kf"],ret["lngDeg_kf"]).mean())

    ret[["latDeg","lngDeg"]] = ret[["latDeg_kf","lngDeg_kf"]]
    
    current_df = ret[cols]

## 道路の一部のみにカルマンフィルタ実行

In [ ]:
GPS_TIME_DEFAULT_TIME = -1
@noglobal(excepts=["GPS_TIME_DEFAULT_TIME"])
def _merge_points(df,startPoint_gps,endPoint_gsp,distance_threshold=0.0):

        
    tmp_df = df.copy();
    tmp_df["millisSinceGpsEpoch_"] = tmp_df["millisSinceGpsEpoch"]//1000

    if (df["phone"].unique().shape[0] != 1):
        path_lists = "\n".join(df["phone"].unique().tolist())
        raise Exception(f"multiple path was found, we found following paths:\n{path_lists}");
        
    if (startPoint_gps == GPS_TIME_DEFAULT_TIME and endPoint_gsp == GPS_TIME_DEFAULT_TIME):                
        return tmp_df;                
    elif (startPoint_gps >0 and endPoint_gsp > 0):
        target_columns = ["latDeg","lngDeg"]

        ## スタートから途中まで
        mean_latdeg,mean_lngdeg = tmp_df[tmp_df["millisSinceGpsEpoch_"] < startPoint_gps][target_columns].mean()
        tmp_df.loc[tmp_df["millisSinceGpsEpoch_"] < startPoint_gps, target_columns] = [mean_latdeg,mean_lngdeg]

        ## 途中からエンドポイントまで
        mean_latdeg,mean_lngdeg= tmp_df[tmp_df["millisSinceGpsEpoch_"] > endPoint_gsp][target_columns].mean()        
        tmp_df.loc[tmp_df["millisSinceGpsEpoch_"] > endPoint_gsp,target_columns] = [mean_latdeg,mean_lngdeg];

        dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],df["latDeg"],df["lngDeg"]).mean()

        if (dis > distance_threshold):
            tmp_df = df.copy();    
    else:
        raise Exception(f"miss parameter specification, \nstartPoint_GPS_time: {startPoint_gps}\nendPoint_GPS_time: {endPoint_gsp}");

    return tmp_df

In [ ]:
from external_lib.gnss_manager import calc_acce_direction
from external_lib.merge_start_end_points import find_first_and_last_stopping_time


@noglobal()
def kalman_all(df):

    if (df["phone"].unique().shape[0] != 1):
        raise Exception()
    
    tmp_df = df.copy()
        
    num = tmp_df[["latDeg","lngDeg"]].to_numpy()                        
    kf = generate_kalmanfilter()  
    result = apply_kalmanfilter(num,kf)           
    tmp_df[["latDeg","lngDeg"]] = result         
       
    return tmp_df


@noglobal(excepts=["calc_acce_direction","find_first_and_last_stopping_time"])
def kalman_patial(df):

    if (df["phone"].unique().shape[0] != 1):
        raise Exception()
    
    tmp_df = df.copy()
    
    path,phone_name = tmp_df["phone"].unique()[0].split("_");
    target_col = "x_f"

    acce_df = calc_acce_direction(path,phone_name);            
    startPoint_gps,endPoint_gsp = find_first_and_last_stopping_time(acce_df,target_col);                        
    
    tmp = tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] > endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] < startPoint_gps)]
    display(tmp)
    
    num = tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] <= endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] >= startPoint_gps) ,["latDeg","lngDeg"]].to_numpy()                        
    kf = generate_kalmanfilter()  
    result = apply_kalmanfilter(num,kf)           
    tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] <= endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] >= startPoint_gps) ,["latDeg","lngDeg"]] = result                                                
    
    return tmp_df

@noglobal(excepts=["calc_acce_direction","find_first_and_last_stopping_time"])
def kalman_all_with_merge(df):

    if (df["phone"].unique().shape[0] != 1):
        raise Exception()
        print(p)
    
    tmp_df = df.copy()
    
    path,phone_name = tmp_df["phone"].unique()[0].split("_");
    target_col = "x_f"

    acce_df = calc_acce_direction(path,phone_name);            
    first_gps,last_gps = find_first_and_last_stopping_time(acce_df,target_col);                        
    
    tmp_df = _merge_points(tmp_df,first_gps,last_gps);
    
    num = tmp_df[["latDeg","lngDeg"]].to_numpy()                        
    kf = generate_kalmanfilter()  
    result = apply_kalmanfilter(num,kf)           
    tmp_df[["latDeg","lngDeg"]] = result         
                        
    return tmp_df

@noglobal(excepts=["calc_acce_direction","find_first_and_last_stopping_time"])
def kalman_partial_with_merge(df):

    if (df["phone"].unique().shape[0] != 1):
        raise Exception()
        print(p)
    
    tmp_df = df.copy()
    
    path,phone_name = tmp_df["phone"].unique()[0].split("_");
    target_col = "x_f"

    acce_df = calc_acce_direction(path,phone_name);            
    startPoint_gps,endPoint_gsp = find_first_and_last_stopping_time(acce_df,target_col);                        
    
    tmp_df = _merge_points(tmp_df,startPoint_gps,endPoint_gsp);
    tmp = tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] > endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] < startPoint_gps)]
    display(tmp)
    
        
    num = tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] <= endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] >= startPoint_gps) ,["latDeg","lngDeg"]].to_numpy()                        
    kf = generate_kalmanfilter()  
    result = apply_kalmanfilter(num,kf)           
    tmp_df.loc[ (tmp_df["millisSinceGpsEpoch_"] <= endPoint_gsp) | (tmp_df["millisSinceGpsEpoch_"] >= startPoint_gps) ,["latDeg","lngDeg"]] = result                                                
                        
    return tmp_df


In [ ]:
list_original = []
list_all = []
list_patial = [];
list_all_with_merge = []
list_partial_with_merge = []


for key,each_df in tqdm(train_df.groupby("phone")):
    
    each_df["millisSinceGpsEpoch_"] = each_df["millisSinceGpsEpoch"] //1000
    
    #kalman_all_df = kalman_all(each_df.copy());
    kalman_patial_df = kalman_patial(each_df);
    #kalman_all_with_merge_df = kalman_all_with_merge(each_df);
    kalman_partial_with_merge_df = kalman_partial_with_merge(each_df);
    
    
    tmp_df = each_df
    dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean()
    list_original.append(dis)
    
    #tmp_df = kalman_all_df
    dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean()
    list_all.append(dis)
    
    #tmp_df = kalman_patial_df
    dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean()
    list_patial.append(dis)
    
    #tmp_df = kalman_all_with_merge_df
    dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean()
    list_all_with_merge.append(dis)
    
    #tmp_df = kalman_partial_with_merge_df
    dis = calc_haversine(tmp_df["latDeg"],tmp_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean()
    list_partial_with_merge.append(dis)
    
    break;
        

In [ ]:
import numpy as np 

list_original
#list_all
#list_patial
#list_all_with_merge 
#list_partial_with_merge 

#print(np.mean(list_all))
print(np.mean(list_patial))
#print(np.mean(list_all_with_merge ))
print(np.mean(list_partial_with_merge ))

#print(list_all)
#print(list_patial)
#print(list_all_with_merge )
#print(list_partial_with_merge )

#display(kalman_all_df.iloc[100]);
#display(kalman_patial_df.iloc[100]);
#display(kalman_all_with_merge_df.iloc[100]);
#display(kalman_partial_with_merge_df.iloc[100]);
